In [44]:
# Mount Colab
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [9]:
# Install pytorch using wheel
!pip3 install torch torchvision

    100% |████████████████████████████████| 519.5MB 26kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x58eec000 @  0x7fc1590712a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
    100% |████████████████████████████████| 61kB 20.8MB/s 
    100% |████████████████████████████████| 2.0MB 4.7MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [45]:
# Install the github repo
!git clone https://github.com/jaderabbit/UnsupervisedMT.git


fatal: destination path 'UnsupervisedMT' already exists and is not an empty directory.


In [47]:
!cd UnsupervisedMT; git pull origin master

remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 8 (delta 6), reused 8 (delta 6), pack-reused 0
Unpacking objects: 100% (8/8), done.
From https://github.com/jaderabbit/UnsupervisedMT
 * branch            master     -> FETCH_HEAD
   0705b7c..fa7c72d  master     -> origin/master
Updating 0705b7c..fa7c72d
Fast-forward
 NMT/src/utils.py | 6 +++---
 1 file changed, 3 insertions(+), 3 deletions(-)


In [17]:
# cd into github repo
# Run preparation
!cd UnsupervisedMT/NMT; ./get_my_data.sh zu https://github.com/LauraMartinus/ukuxhumana/raw/master/leipzig/web_2013_100K_mono.zu


===== Input parameters
 source: en
 target: zu
 target monolingual url: https://github.com/LauraMartinus/ukuxhumana/raw/master/leipzig/web_2013_100K_mono.zu
 
Cloning Moses from GitHub repository...
Cloning into 'mosesdecoder'...
remote: Enumerating objects: 147198, done.
remote: Total 147198 (delta 0), reused 0 (delta 0), pack-reused 147198
Receiving objects: 100% (147198/147198), 129.67 MiB | 20.61 MiB/s, done.
Resolving deltas: 100% (113765/113765), done.
Moses found in: /content/UnsupervisedMT/NMT/tools/mosesdecoder
Cloning fastBPE from GitHub repository...
Cloning into 'fastBPE'...
remote: Enumerating objects: 24, done.
remote: Total 24 (delta 0), reused 0 (delta 0), pack-reused 24
Unpacking objects: 100% (24/24), done.
fastBPE found in: /content/UnsupervisedMT/NMT/tools/fastBPE
Compiling fastBPE...
fastBPE compiled in: /content/UnsupervisedMT/NMT/tools/fastBPE/fast
Cloning fastText from GitHub repository...
Cloning into 'fastText'...
remote: Enumerating objects: 91, done.
remote

In [0]:
# Save weights to drive
file_path='/content/UnsupervisedMT/NMT/data/mono/all.en-zu.60000.vec'
destination_path='/content/gdrive/My Drive/NMT/all.en-zu.60000.vec'
with open(destination_path, 'w') as f:
  with open(file_path,'r') as to_move:
    f.write(to_move.read())


In [28]:
#!/bin/bash
## With
#usage: run_unsupervised_nmt.sh 
!cd UnsupervisedMT/NMT; ./run_unsupervised_nmt.sh zu "/content/gdrive/My Drive/NMT/"


===== Input parameters =====
 TARGET       : zu
 MONO_DATASET : en:./data/mono/mono.en.tok.60000.pth,,;zu:./data/mono/mono.zu.tok.60000.pth,,
 PARA_DATASET : en-zu:,./data/para/en.dev.en.60000.pth,./data/para/en.dev.zu.60000.pth
 PRETRAINED   : ./data/mono/all.en-zu.60000.vec
 
Traceback (most recent call last):
  File "main.py", line 236, in <module>
    check_all_data_params(params)
  File "/content/UnsupervisedMT/NMT/src/data/loader.py", line 341, in check_all_data_params
    assert os.path.isfile(valid_path.replace('XX', lang1))
AssertionError


In [49]:
!cd UnsupervisedMT/NMT; python -u main.py --exp_name enzu     --transformer True  --n_enc_layers 4 --n_dec_layers 4      --share_enc 3  --share_dec 3  --share_lang_emb True  --share_output_emb True     --langs "en,zu" --n_mono -1  --mono_dataset "en:./data/mono/mono.en.tok.60000.pth,,;zu:./data/mono/mono.zu.tok.60000.pth,,"  --para_dataset "en-zu:,./data/para/enzu_parallel.dev.en.60000.pth,./data/para/enzu_parallel.dev.zu.60000.pth"     --mono_directions "en,zu"  --word_shuffle 3 --word_dropout 0.1 --word_blank 0.2     --pivo_directions "en-zu-en,zu-en-zu"     --pretrained_emb "./data/mono/all.en-zu.60000.vec"  --pretrained_out True     --lambda_xe_mono '0:1,100000:0.1,300000:0' --lambda_xe_otfd 1     --otf_num_processes 30   --otf_sync_params_every 1000     --enc_optimizer adam,lr=0.0001     --group_by_size True     --batch_size 32     --epoch_size 500000 --stopping_criterion bleu_en_zu_valid,10     --freeze_enc_emb False --freeze_dec_emb False     --dump_path "/content/gdrive/My Drive/NMT"        


INFO - 11/21/18 19:39:02 - 0:00:00 - ============ Initialized logger ============
INFO - 11/21/18 19:39:02 - 0:00:00 - attention: True
                                     attention_dropout: 0
                                     back_dataset: {}
                                     back_directions: []
                                     batch_size: 32
                                     beam_size: 0
                                     clip_grad_norm: 5
                                     command: python main.py --exp_name 'enzu' --transformer 'True' --n_enc_layers '4' --n_dec_layers '4' --share_enc '3' --share_dec '3' --share_lang_emb 'True' --share_output_emb 'True' --langs 'en,zu' --n_mono '-1' --mono_dataset 'en:./data/mono/mono.en.tok.60000.pth,,;zu:./data/mono/mono.zu.tok.60000.pth,,' --para_dataset 'en-zu:,./data/para/enzu_parallel.dev.en.60000.pth,./data/para/enzu_parallel.dev.zu.60000.pth' --mono_directions 'en,zu' --word_shuffle '3' --word_dropout '0.1' --word_blank '0.2'

In [0]:
os.path.isdir("/content/gdrive/My Drive/")
!mkdir /content/gdrive/My\ Drive/test